# **StereoCamera cone detection**

## **Setup**

Setup the repo and the model.

### **Clone the repositories and import the libraries**

Clone the repository from the smart application course organization and the one for the yolov5 model and import the libraries needed to work with the porject.

In [1]:
!git clone "https://github.com/unipi-smartapp-2021/sensory-cone-detection"   # clone our repository
!git clone "https://github.com/ultralytics/yolov5"  # clone the model repository

Cloning into 'sensory-cone-detection'...
remote: Enumerating objects: 2899, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2899 (delta 15), reused 0 (delta 0), pack-reused 2865
Receiving objects: 100% (2899/2899), 121.76 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (515/515), done.
Cloning into 'yolov5'...
remote: Enumerating objects: 10488, done.
remote: Total 10488 (delta 0), reused 0 (delta 0), pack-reused 10488
Receiving objects: 100% (10488/10488), 10.70 MiB | 12.47 MiB/s, done.
Resolving deltas: 100% (7243/7243), done.


Import the libraries and mount the drive.

In [2]:
pip install py7zr

     |████████████████████████████████| 68 kB 3.0 MB/s 
     |████████████████████████████████| 126 kB 9.0 MB/s 
     |████████████████████████████████| 2.4 MB 32.7 MB/s 
     |████████████████████████████████| 2.0 MB 40.4 MB/s 
     |████████████████████████████████| 357 kB 51.6 MB/s 
     |████████████████████████████████| 48 kB 5.5 MB/s 


In [3]:
import cv2
from tqdm import tqdm
import numpy as np
import py7zr
import zipfile
from google.colab import drive
drive.mount('/content/drive')  # mount the drive

Mounted at /content/drive


### **Manage the dataset**

There are two datasets, choose wisely:  
- the first dataset is the small one and contains 101 images.
- the second one contains more than 8000 images.
- there will be more datasets in the future, or build one yourself.

In [4]:
import zipfile
def extract7z(zip_path):
    with py7zr.SevenZipFile(zip_path, mode='r') as z:
        z.extractall()

def extractzip(zip_path):
    with zipfile.ZipFile(zip_path, mode='r') as z:
        z.extractall()

In [5]:
datasets = ["/content/drive/Shareddrives/Sensory_data/ConeDataset.7z",
            "/content/drive/Shareddrives/Sensory_data/TRset-20211117T155203Z-001.zip",
            "/content/drive/Shareddrives/Sensory_data/Recordings/cameras.zip"]
datasets_extract = [extract7z, extractzip]

In [6]:
datasets_extract[1](datasets[1])

**Extract the chosen dataset**

Manage the dataset by following what is suggested in https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data chapter 1.3. Keep in mind to modify the directory from which you're moving the images (ConeDataset for the first dataset or TRset for the full one).

You can decide to convert the images to grayscale ones or not, then the dataset is splited into train, validation and test sets.

In [7]:
!python sensory-cone-detection/src/manage_dataset.py --dataset TRset --path sensory-cone-detection/src/datasets/camera_dataset

### **Check the yolov5 model and import the weights**

Check if yolov5 is ready and fine.

In [8]:
%cd yolov5
from yolov5.utils import notebook_init
display = notebook_init()  # checks
%cd ../

YOLOv5 🚀 v6.0-184-g6865d19 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 45.9/78.2 GB disk)
/content


Install yolov5 requirements.

In [9]:
!pip install -r yolov5/requirements.txt

     |████████████████████████████████| 596 kB 4.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Import the yolov5 model, all the available models can be found here: https://github.com/ultralytics/yolov5/releases/.

In [10]:
import requests
url = 'https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt'
r = requests.get(url, allow_redirects=True)
open('yolov5s.pt', 'wb').write(r.content)

14698491

## **Train the model**

Train the model following the guide from https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb.

**Main arguments:**
*   *--img*, image size.
*   *--batch*, batch size.
*   *--epochs*, number of epochs.
*   *--data*, the yaml file that specifies where datasets are.
*   *--weights*, the model weights (you can find them here https://github.com/ultralytics/yolov5/releases).
*   *--cache*, to save a cache file of the train and validation sets.
*   *--project*, where to save the results.

**An example of train call would be:**

!python yolov5/train.py --img img_size --batch batch_size --epochs epochs --data dataset.yaml --weights model --cache --project runs/train








In [ ]:
!python yolov5/train.py --img 640 --batch 64 --epochs 10 --data sensory-cone-detection/src/stereocamera/conedataset_camera.yaml --weights sensory-cone-detection/src/stereocamera/best.pt --cache --project train

Use the tensorboard to see the results.

In [ ]:
# Tensorboard  (optional)
%reload_ext tensorboard
#%load_ext tensorboard
%tensorboard --logdir train

## **Evaluate the model**

Use val.py if you have a test set defined in conedataset.yaml. Remember to use --task test or the model will work on the validation set.

In [21]:
!python yolov5/val.py --weights /content/train/exp5/weights/best.pt --img 640 --conf 0.5 --data sensory-cone-detection/src/stereocamera/conedataset_camera.yaml --task test --project runs/test

val: data=sensory-cone-detection/src/stereocamera/conedataset_camera.yaml, weights=['/content/train/exp5/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.5, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/test, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-184-g6865d19 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/sensory-cone-detection/src/datasets/camera_dataset/test' images and labels...862 found, 6 missing, 0 empty, 1 corrupted: 100% 868/868 [00:02<00:00, 299.15it/s]
test: WARNING: sensory-cone-detection/src/datasets/camera_dataset/images/Aidlingen_07_12_video_first_frame_5.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.6951      1.6958      1.6958      1.6958      1.5159     

## **Inference**

Use detect.py if you have no labeled images or you want to try the model on your custom data.

In [ ]:
!rm -rf detect/exp
!python yolov5/detect.py --weights /content/sensory-cone-detection/src/stereocamera/best.pt --img 640 --conf 0.5 --source /content/b83381bf-ad73-4b73-a674-930f48594e61.jpg --exist-ok --save-txt --project detect --hide-labels --line-thickness 2

detect: weights=['/content/lidar-cone-detection/src/stereocamera/best.pt'], source=/content/b83381bf-ad73-4b73-a674-930f48594e61.jpg, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=detect, name=exp, exist_ok=True, line_thickness=2, hide_labels=True, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-124-g1075488 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/b83381bf-ad73-4b73-a674-930f48594e61.jpg: 640x384 3 oranges, Done. (0.224s)
Speed: 2.4ms pre-process, 224.4ms inference, 12.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to detect/exp
1 labels saved to detect/exp/labels
